# Project 2 - GeoTweet

@Author Jeffery Brown (daddyjab)<br>
@Date 3/25/19<br>
@File ETL_for_GeoTweet


In [1]:
# Dependencies
import tweepy
import json
import time
import os
import pandas as pd
from datetime import datetime 
from dateutil import tz
import requests
from pprint import pprint

# Database dependencies
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, ForeignKey

# API Keys
from api_config import *

# Twitter API
# key_twitter_tweetquestor_consumer_api_key
# key_twitter_tweetquestor_consumer_api_secret_key
# key_twitter_tweetquestor_access_token
# key_twitter_tweetquestor_access_secret_token

# Flickr API
# key_flicker_infoquestor_key
# key_flicker_infoquestor_secret


In [2]:
# Setup the database using SQLAlchemy
Base = declarative_base()

In [3]:
class Location(Base):
    __tablename__ = 'locations'
    
    # Defining the columns for the table 'locations',
    # which will hold all of the locations in the U.S. for which
    # top trends data is available, as well as location specific
    # info like latitude/longitude
    id = Column( Integer, primary_key = True)
    woeid = Column( Integer )
    twitter_country = Column( String(100) )
    tritter_country_code = Column( String(10) )
    twitter_name = Column( String(250) )
    twitter_parentid = Column( Integer )
    twitter_type = Column( String(50) )
    country_name = Column( String(250) )
    country_name_only = Column( String(250) )
    country_woeid = Column( Integer )
    county_name = Column( String(250) )
    county_name_only = Column( String(250) )
    county_woeid = Column( Integer )
    latitude = Column( Float )
    longitude = Column( Float )
    name_full = Column( String(250) )
    name_only = Column( String(250) )
    name_woe = Column( String(250) )
    place_type = Column( String(250) )
    state_name = Column( String(250) )
    state_name_only = Column( String(250) )
    state_woeid = Column( Integer )
    timezone = Column( String(250) )
    
    
class Trend(Base):
    __tablename__ = 'trends'
    
    # Defining the columns for the table 'trends',
    # which will hold all of the top trends associated with
    # locations in the 'locations' table
    id = Column( Integer, primary_key = True)
    woeid = Column( Integer )
    twitter_as_of = Column( String(100) )
    twitter_created_at = Column( String(100) )
    twitter_name = Column( String(250) )
    twitter_tweet_name = Column( String(250) )
    twitter_tweet_promoted_content = Column( String(250) )
    twitter_tweet_query = Column( String(250) )
    twitter_tweet_url = Column( String(250) )
    twitter_tweet_volume = Column( Float )
   

In [4]:
# Create an engine that stores data in the local directory's SQLite file
# 'data/twitter_trends.db'.
# NOTE: Since this Jupyter notebook  is running in the ./resources folder
# the path to the database is a little different
db_path_jupyter_notebook = "sqlite:///../data/twitter_trends.db"

# But the Flask app will run from the main folder
db_path_flask_app = "sqlite:///data/twitter_trends.db"

engine = create_engine(db_path_jupyter_notebook)

In [5]:
# Create all table in the engine
# (The equivalent of Create Table statements in raw SQL)
Base.metadata.create_all(engine)

In [6]:
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)

In [7]:
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

In [9]:
# Setup Tweepy API Authentication to access Twitter
auth = tweepy.OAuthHandler(key_twitter_tweetquestor_consumer_api_key, key_twitter_tweetquestor_consumer_api_secret_key)
auth.set_access_token(key_twitter_tweetquestor_access_token, key_twitter_tweetquestor_access_secret_token)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [10]:
def api_calls_remaining( a_api, a_type = "place"):
# Return the number of Twitter API calls remaining
# for the specified API type:
# 'place': Top 10 trending topics for a WOEID
# 'closest': Locations near a specificed lat/long for which Twitter has trending topic info
# 'available': Locations for which Twitter has topic info

    # Get Twitter rate limit information using the Tweepy API
    rate_limits = a_api.rate_limit_status()
    
    # Focus on the rate limits for trends calls
    trends_limits = rate_limits['resources']['trends']
    
    # Return the remaining requests available for the
    # requested type of trends query (or "" if not a valid type)
    try:
        remaining = trends_limits[ f"/trends/{a_type}" ]['remaining']
    except:
        return ""
        
    return remaining

In [11]:
def api_time_before_reset( a_api, a_type = "place"):
# Return the number of minutes until the Twitter API is reset
# for the specified API type:
# 'place': Top 10 trending topics for a WOEID
# 'closest': Locations near a specificed lat/long for which Twitter has trending topic info
# 'available': Locations for which Twitter has topic info

    # Get Twitter rate limit information using the Tweepy API
    rate_limits = a_api.rate_limit_status()
    
    # Focus on the rate limits for trends calls
    trends_limits = rate_limits['resources']['trends']
    
    
    # Return the reset time for the
    # requested type of trends query (or "" if not a valid type)
    try:
        reset_ts = trends_limits[ f"/trends/{a_type}" ]['reset']
    except:
        return -1
        
    # Calculate the remaining time using datetime methods to
    # get the UTC time from the POSIX timestamp
    reset_utc = datetime.utcfromtimestamp(reset_ts)
    
    # Current the current time
    current_utc = datetime.utcnow()
    
    # Calculate the number of seconds remaining,
    # Assumption: reset time will be >= current time
    time_before_reset = (reset_utc - current_utc).total_seconds() / 60.0
    
    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    reset_utc = reset_utc.replace(tzinfo = tz.tzutc() )
    
    # Convert time zone
    reset_local = reset_utc.astimezone( tz.tzlocal() )

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    current_utc = current_utc.replace(tzinfo = tz.tzutc() )
    
    # Convert time zone
    current_local = current_utc.astimezone( tz.tzlocal() )
    print(f"Time Before Reset: {time_before_reset:.1f}: Reset Time: {reset_local.strftime('%Y-%m-%d %H:%M:%S')}, Local Time: {current_local.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Return the time before reset (in minutes)
    return time_before_reset

In [59]:
calls_remaining = api_calls_remaining( api, "available" )
print (f"Trends/Available API call has {calls_remaining} calls remaining")
time_before_reset = api_time_before_reset( api, "available")

Trends/Available API call has 75 calls remaining
Time Before Reset: 15.0: Reset Time: 2019-03-27 01:16:06, Local Time: 2019-03-27 01:01:05


In [60]:
calls_remaining = api_calls_remaining( api, "place" )
print (f"Trends/Place API call has {calls_remaining} calls remaining")
time_before_reset = api_time_before_reset( api, "place")

Trends/Place API call has 75 calls remaining
Time Before Reset: 15.0: Reset Time: 2019-03-27 01:16:09, Local Time: 2019-03-27 01:01:08


In [14]:
def get_trends_available_to_df( ):
# Get locations that have trends data from a api.trends_available() call,
# flatten the data, and create a dataframe

    # Obtain the WOEID locations for which Twitter Trends info is available
    try:
        trends_avail = api.trends_available()
        
    except TweepError as e:
        # No top trends info available for this WOEID, return False
        print(f"Error obtaining top trends for WOEID {a_woeid}: ", e)
        return False
    
    # Import trend availability info into a dataframe
    trends_avail_df = pd.DataFrame.from_dict(trends_avail, orient='columns')

    # Retain only locations in the U.S.
    trends_avail_df = trends_avail_df[ (trends_avail_df['countryCode'] == "US") ]
        
    # Reset the index
    trends_avail_df.reset_index(drop=True, inplace=True)

    # Flatten the dataframe by unpacking the placeType column information into separate columns
    trends_avail_df['twitter_type'] = trends_avail_df['placeType'].map( lambda x: x['name'])

    # Remove unneeded fields
    trends_avail_df.drop(['placeType', 'url' ], axis='columns' , inplace = True)

    # Rename the fields
    trends_avail_df.rename(columns={
        'woeid': 'woeid',
        'country': 'twitter_country',
        'countryCode': 'tritter_country_code',
        'name': 'twitter_name',
        'parentid': 'twitter_parentid' }, inplace=True)
    
    return trends_avail_df

In [15]:
# Flatten the Twitter Trends results and populate in a Dataframe
trends_available_df = get_trends_available_to_df()

In [16]:
trends_available_df.head()

,twitter_country,tritter_country_code,twitter_name,twitter_parentid,woeid,twitter_type
0,United States,US,Albuquerque,23424977,2352824,Town
1,United States,US,Atlanta,23424977,2357024,Town
2,United States,US,Austin,23424977,2357536,Town
3,United States,US,Baltimore,23424977,2358820,Town
4,United States,US,Baton Rouge,23424977,2359991,Town


In [17]:
def get_location_info( a_woeid ):
# Use Flickr API call to get location information associated with a Yahoo! WOEID
# Note: Yahoo! no longer supports this type of lookup! :(

    # Setup the Flickr API base URL
    flickr_api_base_url = f"https://api.flickr.com/services/rest/?method=flickr.places.getInfo&api_key={key_flicker_infoquestor_key}&format=json&nojsoncallback=1&woe_id="

    # Populate the WOEID and convert to string format
    woeid_to_search = str(a_woeid)
    
    # Build the full URL for API REST request
    flickr_api_url = flickr_api_base_url + woeid_to_search

    try:
        # Get the REST response, which will be in JSON format
        response = requests.get(url=flickr_api_url)
        
    except requests.exceptions.RequestException as e:
        print("Error obtaining location information for WOEID {a_woeid}: ", e)
        return False
    
    # Parse the json
    location_data = response.json()
    
    # Check for failure to locate the information
    if (location_data['stat'] == 'fail'):
        print(f"Error finding location WOEID {a_woeid}: {location_data['message']}")
        
        
    #pprint(location_data)
    
    # Return just a useful subset of the location info as flattened dictionary
    key_location_info = {}
    
    # Basic information that should be present for any location
    try:
        key_location_info.update( {
            'woeid': int(location_data['place']['woeid']),
            'name_woe': location_data['place']['woe_name'],
            'name_full': location_data['place']['name'],
            'name_only': location_data['place']['name'].split(",")[0].strip(),
            'place_type': location_data['place']['place_type'],
            'latitude': float(location_data['place']['latitude']),
            'longitude': float(location_data['place']['longitude']),
        })
                
    except:
        print("Error - basic location information not returned for WOEID{a_woeid}: ", sys.exc_info()[0])
    
    # Timezone associated with the location - if available
    try:
        key_location_info.update( {
            'timezone': location_data['place']['timezone']  
        })
        
    except:
        key_location_info.update( {
            'timezone': None
        })
        
    # County associated with the location - if available
    try:
        key_location_info.update( {
            'county_name': location_data['place']['county']['_content'],
            'county_name_only': location_data['place']['county']['_content'].split(",")[0].strip(),
            'county_woeid': int(location_data['place']['county']['woeid']),
        })
    except:
        key_location_info.update( {
            'county_name': None,
            'county_name_only': None,
            'county_woeid': None,
        })
        
    # State associated with the location - if available
    try:
        key_location_info.update( {
            'state_name': location_data['place']['region']['_content'],
            'state_name_only': location_data['place']['region']['_content'].split(",")[0].strip(),
            'state_woeid': int(location_data['place']['region']['woeid']),
        })
    except:
        key_location_info.update( {
            'state_name': None,
            'state_name_only': None,
            'state_woeid': None,
        })
        
    # Country associated with the location - if available
    try:
        key_location_info.update( {
            'country_name': location_data['place']['country']['_content'],
            'country_name_only': location_data['place']['country']['_content'].split(",")[0].strip(),
            'country_woeid': int(location_data['place']['country']['woeid']),
        })
    except:
        key_location_info.update( {
            'country_name': None,
            'country_name_only': None,
            'country_woeid': None, 
        })
    
    return key_location_info

In [45]:
# Use the get_location_info() function to add location info (from Flickr)
# for each location (Twitter WOEID) that has trend info
#loc_info_list =  list( trends_available_df['woeid'][0:2].apply( get_location_info ) )
loc_info_list =  list( trends_available_df['woeid'].apply( get_location_info ) )

In [46]:
print(loc_info_list)

[{'woeid': 2352824, 'name_woe': 'Albuquerque', 'name_full': 'Albuquerque, New Mexico, United States', 'name_only': 'Albuquerque', 'place_type': 'locality', 'latitude': 35.105, 'longitude': -106.647, 'timezone': 'America/Denver', 'county_name': 'Bernalillo County, New Mexico, United States', 'county_name_only': 'Bernalillo County', 'county_woeid': 12589279, 'state_name': 'New Mexico, United States', 'state_name_only': 'New Mexico', 'state_woeid': 2347590, 'country_name': 'United States', 'country_name_only': 'United States', 'country_woeid': 23424977}, {'woeid': 2357024, 'name_woe': 'Atlanta', 'name_full': 'Atlanta, Georgia, United States', 'name_only': 'Atlanta', 'place_type': 'locality', 'latitude': 33.763, 'longitude': -84.423, 'timezone': 'America/New_York', 'county_name': 'Fulton County, Georgia, United States', 'county_name_only': 'Fulton County', 'county_woeid': 12587929, 'state_name': 'Georgia, United States', 'state_name_only': 'Georgia', 'state_woeid': 2347569, 'country_name':

In [47]:
# Create a DataFrame from the location info list
loc_info_df = pd.DataFrame.from_dict(loc_info_list)

In [48]:
loc_info_df

,country_name,country_name_only,country_woeid,county_name,county_name_only,county_woeid,latitude,longitude,name_full,name_only,name_woe,place_type,state_name,state_name_only,state_woeid,timezone,woeid
0,United States,United States,23424977,"Bernalillo County, New Mexico, United States",Bernalillo County,12589279.0,35.105,-106.647,"Albuquerque, New Mexico, United States",Albuquerque,Albuquerque,locality,"New Mexico, United States",New Mexico,2347590.0,America/Denver,2352824
1,United States,United States,23424977,"Fulton County, Georgia, United States",Fulton County,12587929.0,33.763,-84.423,"Atlanta, Georgia, United States",Atlanta,Atlanta,locality,"Georgia, United States",Georgia,2347569.0,America/New_York,2357024
2,United States,United States,23424977,"Travis County, Texas, United States",Travis County,12590233.0,30.306,-97.752,"Austin, Texas, United States",Austin,Austin,locality,"Texas, United States",Texas,2347602.0,America/Chicago,2357536
3,United States,United States,23424977,"Baltimore City County, Maryland, United States",Baltimore City County,12588679.0,39.300,-76.610,"Baltimore, Maryland, United States",Baltimore,Baltimore,locality,"Maryland, United States",Maryland,2347579.0,America/New_York,2358820
4,United States,United States,23424977,"East Baton Rouge Parish County, Louisiana, Uni...",East Baton Rouge Parish County,12588612.0,30.450,-91.126,"Baton Rouge, Louisiana, United States",Baton Rouge,Baton Rouge,locality,"Louisiana, United States",Louisiana,2347577.0,America/Chicago,2359991
5,United States,United States,23424977,"Jefferson County, Alabama, United States",Jefferson County,12587523.0,33.531,-86.823,"Birmingham, Alabama, United States",Birmingham,Birmingham,locality,"Alabama, United States",Alabama,2347559.0,America/Chicago,2364559
6,United States,United States,23424977,"Suffolk County, Massachusetts, United States",Suffolk County,12588712.0,42.358,-71.056,"Boston, Massachusetts, United States",Boston,Boston,locality,"Massachusetts, United States",Massachusetts,2347580.0,America/New_York,2367105
7,United States,United States,23424977,"Mecklenburg County, North Carolina, United States",Mecklenburg County,12589433.0,35.222,-80.837,"Charlotte, North Carolina, United States",Charlotte,Charlotte,locality,"North Carolina, United States",North Carolina,2347592.0,America/New_York,2378426
8,United States,United States,23424977,"Cook County, Illinois, United States",Cook County,12588093.0,41.884,-87.632,"Chicago, Illinois, United States",Chicago,Chicago,locality,"Illinois, United States",Illinois,2347572.0,America/Chicago,2379574
9,United States,United States,23424977,"Hamilton County, Ohio, United States",Hamilton County,12589557.0,39.141,-84.505,"Cincinnati, Ohio, United States",Cincinnati,Cincinnati,locality,"Ohio, United States",Ohio,2347594.0,America/New_York,2380358


In [49]:
# Merge the Twitter trend location available dataframe with the
# location info dataframe to create a master list of all
# Twitter Trend locations and associated location information
twitter_trend_locations_df = trends_available_df.merge(loc_info_df, how='inner', on='woeid')

In [50]:
twitter_trend_locations_df

,twitter_country,tritter_country_code,twitter_name,twitter_parentid,woeid,twitter_type,country_name,country_name_only,country_woeid,county_name,...,latitude,longitude,name_full,name_only,name_woe,place_type,state_name,state_name_only,state_woeid,timezone
0,United States,US,Albuquerque,23424977,2352824,Town,United States,United States,23424977,"Bernalillo County, New Mexico, United States",...,35.105,-106.647,"Albuquerque, New Mexico, United States",Albuquerque,Albuquerque,locality,"New Mexico, United States",New Mexico,2347590.0,America/Denver
1,United States,US,Atlanta,23424977,2357024,Town,United States,United States,23424977,"Fulton County, Georgia, United States",...,33.763,-84.423,"Atlanta, Georgia, United States",Atlanta,Atlanta,locality,"Georgia, United States",Georgia,2347569.0,America/New_York
2,United States,US,Austin,23424977,2357536,Town,United States,United States,23424977,"Travis County, Texas, United States",...,30.306,-97.752,"Austin, Texas, United States",Austin,Austin,locality,"Texas, United States",Texas,2347602.0,America/Chicago
3,United States,US,Baltimore,23424977,2358820,Town,United States,United States,23424977,"Baltimore City County, Maryland, United States",...,39.300,-76.610,"Baltimore, Maryland, United States",Baltimore,Baltimore,locality,"Maryland, United States",Maryland,2347579.0,America/New_York
4,United States,US,Baton Rouge,23424977,2359991,Town,United States,United States,23424977,"East Baton Rouge Parish County, Louisiana, Uni...",...,30.450,-91.126,"Baton Rouge, Louisiana, United States",Baton Rouge,Baton Rouge,locality,"Louisiana, United States",Louisiana,2347577.0,America/Chicago
5,United States,US,Birmingham,23424977,2364559,Town,United States,United States,23424977,"Jefferson County, Alabama, United States",...,33.531,-86.823,"Birmingham, Alabama, United States",Birmingham,Birmingham,locality,"Alabama, United States",Alabama,2347559.0,America/Chicago
6,United States,US,Boston,23424977,2367105,Town,United States,United States,23424977,"Suffolk County, Massachusetts, United States",...,42.358,-71.056,"Boston, Massachusetts, United States",Boston,Boston,locality,"Massachusetts, United States",Massachusetts,2347580.0,America/New_York
7,United States,US,Charlotte,23424977,2378426,Town,United States,United States,23424977,"Mecklenburg County, North Carolina, United States",...,35.222,-80.837,"Charlotte, North Carolina, United States",Charlotte,Charlotte,locality,"North Carolina, United States",North Carolina,2347592.0,America/New_York
8,United States,US,Chicago,23424977,2379574,Town,United States,United States,23424977,"Cook County, Illinois, United States",...,41.884,-87.632,"Chicago, Illinois, United States",Chicago,Chicago,locality,"Illinois, United States",Illinois,2347572.0,America/Chicago
9,United States,US,Cincinnati,23424977,2380358,Town,United States,United States,23424977,"Hamilton County, Ohio, United States",...,39.141,-84.505,"Cincinnati, Ohio, United States",Cincinnati,Cincinnati,locality,"Ohio, United States",Ohio,2347594.0,America/New_York


In [52]:
# Write this table of location data to the database
twitter_trend_locations_df.to_sql( 'locations', con=engine, if_exists='append', index=False)

In [53]:
q_results = session.query(Location).all()

In [54]:
for row in q_results:
    print(row.woeid, row.name_full)

2352824 Albuquerque, New Mexico, United States
2357024 Atlanta, Georgia, United States
2352824 Albuquerque, New Mexico, United States
2357024 Atlanta, Georgia, United States
2359991 Baton Rouge, Louisiana, United States
2364559 Birmingham, Alabama, United States
2367105 Boston, Massachusetts, United States
2378426 Charlotte, North Carolina, United States
2379574 Chicago, Illinois, United States
2380358 Cincinnati, Ohio, United States
2381475 Cleveland, Ohio, United States
2383489 Colorado Springs, Colorado, United States
2383660 Columbus, Ohio, United States
2388929 Dallas, Texas, United States
2391279 Denver, Colorado, United States
2391585 Detroit, Michigan, United States
2397816 El Paso, Texas, United States
2407517 Fresno, California, United States
2414469 Greensboro, North Carolina, United States
2418046 Harrisburg, Pennsylvania, United States
2423945 Honolulu, Hawaii, United States
2424766 Houston, Texas, United States
2427032 Indianapolis, Indiana, United States
2428184 Jackson,

In [25]:
def get_trends_for_loc( a_woeid ):
# Get top Twitter trending tweets for a location specified by a WOEID,
# flatten the data, and return it as a list of dictionaries

    # Import trend availability info into a dataframe
    try:
        top_trends = api.trends_place( a_woeid )[0]
        
    except TweepError as e:
        # No top trends info available for this WOEID, return False
        print(f"Error obtaining top trends for WOEID {a_woeid}: ", e)
        return False
    
    #pprint(top_trends)
    
    # Repeat some information that is common for all elements in the trends list
    common_info = {}
        
    # Basic information that should be present for any location
    # 'as_of': '2019-03-26T21:22:42Z',
    # 'created_at': '2019-03-26T21:17:18Z',
    # 'locations': [{'name': 'Atlanta', 'woeid': 2357024}]
    try:
        common_info.update( {
            'woeid': int(top_trends['locations'][0]['woeid']),
            'twitter_name': top_trends['locations'][0]['name'],
            'twitter_created_at': top_trends['created_at'],
            'twitter_as_of': top_trends['as_of']
        })
                
    except:
        print("Error - basic location information not returned for WOEID{a_woeid}: ", sys.exc_info()[0])
   
    # Loop through all of the trends and store in an array of dictionary elements
    # 'name': 'Jussie Smollett'
    # 'promoted_content': None
    # 'query': '%22Jussie+Smollett%22'
    # 'tweet_volume': 581331
    # 'url': 'http://twitter.com/search?q=%22Jussie+Smollett%22'

    # Return the trends as an array of flattened dictionaries
    trend_info = []

    for ti in top_trends['trends']:
        
        # Put the trend info into a dictionary, starting with the common info
        this_trend = common_info.copy()
        
        # Timezone associated with the location - if available
        try:
            this_trend.update( {
                'twitter_tweet_name': ti['name'],
                'twitter_tweet_promoted_content': ti['promoted_content'],
                'twitter_tweet_query': ti['query'],
                'twitter_tweet_volume': ti['tweet_volume'],
                'twitter_tweet_url': ti['url']
            })

        except:
            this_trend.update( {
                'twitter_tweet_name': None,
                'twitter_tweet_promoted_content': None,
                'twitter_tweet_query': None,
                'twitter_tweet_volume': None,
                'twitter_tweet_url': None
            })
            
        # Append this trend to the list
        trend_info.append( this_trend )
    
    return trend_info

In [26]:
# Get trend info for a WOEID location
t_info = get_trends_for_loc(2357024)
t_info_df = pd.DataFrame.from_dict(t_info)

In [27]:
t_info_df = pd.DataFrame.from_dict(t_info)

In [56]:
t_info_df

,twitter_as_of,twitter_created_at,twitter_name,twitter_tweet_name,twitter_tweet_promoted_content,twitter_tweet_query,twitter_tweet_url,twitter_tweet_volume,woeid
0,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#ThisIsUs,None,%23ThisIsUs,http://twitter.com/search?q=%23ThisIsUs,24521.0,2357024
1,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Asuka,None,Asuka,http://twitter.com/search?q=Asuka,66265.0,2357024
2,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#THWg,None,%23THWg,http://twitter.com/search?q=%23THWg,NaN,2357024
3,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Jussie Smollett,None,%22Jussie+Smollett%22,http://twitter.com/search?q=%22Jussie+Smollett%22,837613.0,2357024
4,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Special Olympics,None,%22Special+Olympics%22,http://twitter.com/search?q=%22Special+Olympic...,231529.0,2357024
5,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Cardi,None,Cardi,http://twitter.com/search?q=Cardi,130663.0,2357024
6,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#TuesdayThoughts,None,%23TuesdayThoughts,http://twitter.com/search?q=%23TuesdayThoughts,130842.0,2357024
7,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#SDLive,None,%23SDLive,http://twitter.com/search?q=%23SDLive,86532.0,2357024
8,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Lance,None,Lance,http://twitter.com/search?q=Lance,103506.0,2357024
9,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#TemptationIsland,None,%23TemptationIsland,http://twitter.com/search?q=%23TemptationIsland,NaN,2357024


In [29]:
t_info = get_trends_for_loc(2352824)

In [30]:
t_info_df = t_info_df.append( pd.DataFrame.from_dict(t_info), ignore_index = True)

In [31]:
t_info_df

,twitter_as_of,twitter_created_at,twitter_name,twitter_tweet_name,twitter_tweet_promoted_content,twitter_tweet_query,twitter_tweet_url,twitter_tweet_volume,woeid
0,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#ThisIsUs,None,%23ThisIsUs,http://twitter.com/search?q=%23ThisIsUs,24521.0,2357024
1,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Asuka,None,Asuka,http://twitter.com/search?q=Asuka,66265.0,2357024
2,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#THWg,None,%23THWg,http://twitter.com/search?q=%23THWg,NaN,2357024
3,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Jussie Smollett,None,%22Jussie+Smollett%22,http://twitter.com/search?q=%22Jussie+Smollett%22,837613.0,2357024
4,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Special Olympics,None,%22Special+Olympics%22,http://twitter.com/search?q=%22Special+Olympic...,231529.0,2357024
5,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Cardi,None,Cardi,http://twitter.com/search?q=Cardi,130663.0,2357024
6,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#TuesdayThoughts,None,%23TuesdayThoughts,http://twitter.com/search?q=%23TuesdayThoughts,130842.0,2357024
7,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#SDLive,None,%23SDLive,http://twitter.com/search?q=%23SDLive,86532.0,2357024
8,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,Lance,None,Lance,http://twitter.com/search?q=Lance,103506.0,2357024
9,2019-03-27T05:19:57Z,2019-03-27T05:16:48Z,Atlanta,#TemptationIsland,None,%23TemptationIsland,http://twitter.com/search?q=%23TemptationIsland,NaN,2357024


In [32]:
t_info_df.dtypes

twitter_as_of                      object
twitter_created_at                 object
twitter_name                       object
twitter_tweet_name                 object
twitter_tweet_promoted_content     object
twitter_tweet_query                object
twitter_tweet_url                  object
twitter_tweet_volume              float64
woeid                               int64
dtype: object

In [68]:
for tw_woeid in twitter_trend_locations_df['woeid']:
    print(tw_woeid)

    # Make sure we haven't hit the rate limit yet
    calls_remaining = api_calls_remaining( api, "place" )
    print (f"Trends/Place API call has {calls_remaining} calls remaining")
    time_before_reset = api_time_before_reset( api, "place")

    # If we're rate limited for the trends/place API,
    # then wait until the next reset =
    # 'time_before_reset' minutes+ 1 minute buffer
    if (calls_remaining < 1):
        print (f"Waiting {time_before_reset} minutes due to rate limit")
        time.sleep( (time_before_reset+1) * 60)
        
    # Get trend info for a WOEID location
    t_info = get_trends_for_loc(tw_woeid)
    
    # Create a DataFrame
    t_info_df = pd.DataFrame.from_dict(t_info)
    
    # Append it to the 'trends' database table
    t_info_df.to_sql( 'trends', con=engine, if_exists='append', index=False)
    
    
    


2352824
Trends/Place API call has 59 calls remaining
Time Before Reset: 8.8: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:13:35
2357024
Trends/Place API call has 58 calls remaining
Time Before Reset: 8.8: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:13:36
2357536
Trends/Place API call has 57 calls remaining
Time Before Reset: 8.7: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:13:37
2358820
Trends/Place API call has 56 calls remaining
Time Before Reset: 8.7: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:13:37
2359991
Trends/Place API call has 55 calls remaining
Time Before Reset: 8.7: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:13:38
2364559
Trends/Place API call has 54 calls remaining
Time Before Reset: 8.7: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:13:39
2367105
Trends/Place API call has 53 calls remaining
Time Before Reset: 8.7: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:13:40
237842

2503713
Trends/Place API call has 1 calls remaining
Time Before Reset: 8.0: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:14:24
2503863
Trends/Place API call has 0 calls remaining
Time Before Reset: 7.9: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:14:25
Waiting 7.946771116666667 minutes due to rate limit
2508428
Trends/Place API call has 74 calls remaining
Time Before Reset: 15.0: Reset Time: 2019-03-27 01:37:23, Local Time: 2019-03-27 01:22:22
2512636
Trends/Place API call has 73 calls remaining
Time Before Reset: 15.0: Reset Time: 2019-03-27 01:37:23, Local Time: 2019-03-27 01:22:23
2514815
Trends/Place API call has 72 calls remaining
Time Before Reset: 15.0: Reset Time: 2019-03-27 01:37:23, Local Time: 2019-03-27 01:22:24
23424977
Trends/Place API call has 71 calls remaining
Time Before Reset: 15.0: Reset Time: 2019-03-27 01:37:23, Local Time: 2019-03-27 01:22:25


In [66]:
calls_remaining = api_calls_remaining( api, "place" )
print (calls_remaining)

59


In [67]:
api_time_before_reset( api, "place")

Time Before Reset: 9.4: Reset Time: 2019-03-27 01:22:22, Local Time: 2019-03-27 01:12:56


9.41939545